In [1]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field, field_validator
import os

In [2]:
load_dotenv()

True

In [3]:
API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_ENDPOINT"] = ENDPOINT
os.environ["AZURE_OPENAI_API_KEY"] = API_KEY

In [4]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",
    api_version="2023-03-15-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [5]:
chat_history = {
    "introduction_state": True,
    "profiling_state": True,
    "preferences_state": False,
    "history": "chat history between investor and ai agent: Hi i want to invest to buy a car! Unfortunately your investment is not SMART! Okay I want to buy a car worth 600000$ in 5 years with initial investment 500000$ and expecting 0.05 return anaually Cool this is smart goal! can you give me your preferences toward any of the assets?"
}

In [9]:
def fallback(llm, query, chat_history):
    router_template = f"""
    You are a fallback message manager, kind and very concise and not talkative.
    You will be provided with the history {chat_history['history']} of a chat between an investor and ai agent, the profiling_state(True or False) {chat_history['profiling_state']}
    the preferences_state(True or False) {chat_history['preferences_state']}.
    if the preferences_state is False and the investor is still giving you other goals, or talking about any other topic instead of giving you their investing preferences! explain to him that you are not tasked to provide him with any insights, you are just here to support him to define a goal and then share with you his investment preferences.
    else if the preferences_state is True, Ask him if he is sure he mentioned all his preferences about asset type, nature, companies, markets he wants to invest in or not to invest in!
    You'll be given also the latest message by the investor to understand better why this message is considered as fallback
    """

    route_prompt = ChatPromptTemplate.from_messages(
        [("system", router_template), ("human", " The latest message by the user :{fallback}")]
    )
    router = route_prompt | llm
    response = router.invoke({"fallback": query})
    return response

In [11]:
msg = fallback(llm, "The car may require a lot of work", chat_history)

In [12]:
msg

AIMessage(content="I understand your concern about the car requiring work. However, I'm here to help you define your investment goal. Please share your investment preferences so we can proceed.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 254, 'total_tokens': 286, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_878413d04d', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-9aa4552b-46d6-4fc0-be54-3ed636b74865-0', usage_metadata={'input_tokens': 254, 'output_tokens': 32, 'total_tokens': 286})